In [ ]:
import os
import struct
import pandas as pd
import numpy as np
import talib as tdx

In [ ]:

def readTdxLdayFile(fname="sh000001.day"):
  dataSet=[]
  with open(fname,'rb') as fl:
    buffer=fl.read() #读取数据到缓存
    size=len(buffer) 
    rowSize=32 #通信达day数据，每32个字节一组数据
    code=os.path.basename(fname).replace('.day','')
    for i in range(0,size,rowSize): #步长为32遍历buffer
      row=list( struct.unpack('IIIIIfII',buffer[i:i+rowSize]) )
      row[1]=row[1]/100
      row[2]=row[2]/100
      row[3]=row[3]/100
      row[4]=row[4]/100
      row.pop() #移除最后无意义字段
      row.insert(0,code)
      dataSet.append(row) 

  data=pd.DataFrame(data=dataSet,columns=['code','tradeDate','open','high','low','close','amount','vol'])
#   data=data.set_index(['tradeDate'])
  return data


In [115]:

def readTdxLdayPath(path):
  files = os.listdir(path)
  codes=[]
  dataSet=[]
  for i in range(0,len(files)):
    fname = os.path.join(path,files[i])
    if os.path.isdir(fname):
      continue
    with open(fname,'rb') as fl:
      buffer=fl.read() #读取数据到缓存
      size=len(buffer) 
      rowSize=32 #通信达day数据，每32个字节一组数据
      code=os.path.basename(fname).replace('.day','')
      for i in range(0,size,rowSize): #步长为32遍历buffer
        row=list( struct.unpack('IIIIIfII',buffer[i:i+rowSize]) )
        row[1]=row[1]/100
        row[2]=row[2]/100
        row[3]=row[3]/100
        row[4]=row[4]/100
        row.pop() #移除最后无意义字段
        row.insert(0,code)
        dataSet.append(row) 
      codes.append(code)
  data=pd.DataFrame(data=dataSet,columns=['code','tradeDate','open','high','low','close','amount','vol'])
  data=data.set_index(['code','tradeDate'])
  return codes, data


In [116]:
codes,data_df=readTdxLdayPath('data') #读取目录下面的所有文件

In [117]:
data_df

open     high      low    close        amount  \
code     tradeDate                                                     
sh000002 19901219     96.05    99.98    95.79    99.98  4.940000e+05   
         19901220    104.30   104.39    99.98   104.39  8.400000e+04   
         19901221    109.07   109.13   103.73   109.13  1.600000e+04   
         19901224    113.57   114.55   109.13   114.55  3.100000e+04   
         19901225    120.09   120.25   114.55   120.25  6.000000e+03   
         19901226    125.27   125.27   120.25   125.27  5.300000e+04   
         19901227    125.27   125.28   125.27   125.28  1.040000e+05   
         19901228    126.39   126.45   125.28   126.45  8.800000e+04   
         19901231    126.56   127.61   126.48   127.61  6.000000e+04   
         19910102    127.61   128.84   127.61   128.84  5.900000e+04   
         19910103    128.84   130.14   128.84   130.14  9.300000e+04   
         19910104    131.27   131.44   130.14   131.44  2.610000e+05   
         19910107    131.99   132.06   131.45   132.06  1.410000e+05   
         19910108    132.62   132.68   132.06   132.68  1.806000e+06   
         19910109    133.30   133.34   132.68   133.34  3.228000e+06   
         19910110    133.93   133.97   133.34   133.97  5.399000e+06   
         19910111    134.61   134.61   134.51   134.60  7.115000e+06   
         19910114    134.11   135.19   134.11   134.67  6.883000e+06   
         19910115    134.21   134.74   134.19   134.74  1.010000e+06   
         19910116    134.19   134.74   134.14   134.24  2.700000e+05   
         19910117    133.67   134.25   133.65   134.25  3.340000e+05   
         19910118    133.70   134.25   133.67   134.24  1.570000e+06   
         19910121    133.70   134.24   133.66   134.24  1.029000e+06   
         19910122    133.72   134.24   133.66   133.72  1.800000e+05   
         19910123    133.17   133.72   133.14   133.17  5.750000e+05   
         19910124    132.61   133.17   132.57   132.61  9.170000e+05   
         19910125    132.05   132.07   132.03   132.05  1.722000e+06   
         19910128    131.46   131.55   131.46   131.46  2.565000e+06   
         19910129    130.95   130.97   130.95   130.95  7.100000e+05   
         19910130    130.44   130.95   130.41   130.44  2.600000e+05   
...                     ...      ...      ...      ...           ...   
sh000003 20190507   2914.29  2937.28  2889.07  2926.39  2.454442e+11   
         20190508   2873.14  2929.43  2866.70  2893.76  2.188926e+11   
         20190509   2871.20  2888.86  2845.04  2850.95  1.974469e+11   
         20190510   2878.23  2941.45  2838.38  2939.21  2.741763e+11   
         20190513   2905.07  2921.41  2892.17  2903.71  2.026241e+11   
         20190514   2872.83  2909.20  2872.83  2883.61  2.012233e+11   
         20190515   2902.64  2945.39  2902.64  2938.68  2.262628e+11   
         20190516   2933.50  2956.17  2929.06  2955.71  2.262031e+11   
         20190517   2955.77  2956.78  2873.80  2882.30  2.478752e+11   
         20190520   2874.80  2882.63  2838.45  2870.60  1.992108e+11   
         20190521   2867.71  2919.24  2862.50  2905.97  2.027778e+11   
         20190522   2905.52  2912.40  2879.64  2891.70  1.936294e+11   
         20190523   2880.84  2885.14  2846.96  2852.52  1.928695e+11   
         20190524   2847.84  2871.86  2846.02  2853.00  1.626893e+11   
         20190527   2851.28  2898.13  2833.04  2892.38  1.937687e+11   
         20190528   2890.27  2924.04  2887.08  2909.91  2.272538e+11   
         20190529   2894.83  2934.98  2890.66  2914.70  2.028550e+11   
         20190530   2903.42  2907.85  2881.38  2905.81  1.971055e+11   
         20190531   2904.50  2922.91  2895.58  2898.70  1.912804e+11   
         20190603   2901.74  2920.83  2875.90  2890.08  2.080733e+11   
         20190604   2887.64  2888.39  2851.97  2862.28  1.811189e+11   
         20190605   2882.94  2888.77  2858.57  2861.42  1.752008e+11   
         20190606   2862.33  2862.33  2822.19  2827.80  1.64850

In [ ]:
da=data_df.reset_index('tradeDate')

In [ ]:
import datetime

In [ ]:
pd.to_datetime(da.tradeDate)

In [ ]:
# data_df.to_csv('test.csv')

In [ ]:
data_df.index[,-1:-1]

In [ ]:
def select1(code,data):
    # 连续三日缩量
    ch= data.close.iloc[-1] * 1.1
    cl= data.close.iloc[-1] * 0.9
#     ch= data.close * 1.1
#     cl = data.close * 0.9
    
    df=pd.concat([tdx.MA(data.close, x) for x in (5,10,20,30,60,90,120,250) ], axis = 1).dropna()[-1:]
    df.columns = [u'm5',u'm10',u'm20',u'm30',u'm60',u'm90',u'm120', u'm250']  
    df_h = df.apply(lambda x:x.max() <= ch,  axis = 1 )
    df_l = df.apply(lambda x:x.min() >= cl,  axis = 1 )
    
    df['dfh'] = df_h
    df['dfl'] = df_l
    df['code'] =code
#     out=df.iloc[-1].apply(lambda x: True if x>cl and x < ch else False)
    df=df.reset_index('tradeDate')
    df=df.set_index(['code','tradeDate'])
    return df


In [ ]:
bbb=select1('sh000001',data_df.loc['sh000001',])

In [ ]:
bbb

In [ ]:
bbb=bbb.set_index(['code','tradeDate'])

In [ ]:
data=bbb.set_index(['code','tradeDate'])

In [ ]:
output=None
for code in codes:
  aaa=data_df.loc[code,]
  out=select1(code, aaa)
  if output is None:
    output = out
  else:
#     print(code)
    output=output.append(out)

In [ ]:
output

In [ ]:
output.query('dfh==True and dfl==True').to_csv('out1.csv')

In [ ]:
bb=select1('000001',aaa)

In [ ]:
type(bb)

In [ ]:
import talib as tdx

In [ ]:
aaa=pd.read_csv('test.csv')

In [ ]:
aaa.set_index('vol').sort_index()

In [ ]:
df=readTdxLdayFile()
df['mon'] = df.tradeDate.apply(lambda x : str(x)[0:6])
df=df.set_index(['tradeDate'])

In [ ]:
dfmax=df.groupby(['mon']).apply(lambda x: x[x.high ==x.high.max()])
dfmax.drop_duplicates(subset=['high','mon'],keep='first',inplace=True)
dfmin=df.groupby(['mon']).apply(lambda x: x[x.low ==x.low.min()])
dfmin.drop_duplicates(subset=['low','mon'],keep='first',inplace=True)

In [ ]:
dfmax.to_csv('max.csv')

In [ ]:
dfmin.to_csv('min.csv')

In [ ]:
dfmax

In [ ]:
for x in dfmax.index:
  print(df.loc[x[1]])